# component_from_yaml 

You can define a component using a YAML netlist.

YAML -> Component


## instances

Lets start by defining the `instances` in YAML

In [ ]:
import pp

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 10
    mmi_short:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
"""

c = pp.component_from_yaml(yaml)
c

In [ ]:
c.instances

You can modify the instances later (not recommended).

I recommend that you define that using `placements` in the `YAML` file

In [ ]:
c.instances['mmi_long'].x = 100
c.show()
c.plot()

## placements

Lets define the placecemts of each instance by defining a `placements` section in YAML

Lets place an `mmi_long` where you can place the `W0` port at `x=20, y=10`

In [ ]:
import pp

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
placements:        
    mmi_long:
        port: W0
        x: 20
        y: 10
"""

c = pp.component_from_yaml(yaml)
c.show()
c.plot()

You can also mirror it

In [ ]:
import pp

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
placements:        
    mmi_long:
        port: W0
        x: 20
        y: 10
        mirror: True
"""

c = pp.component_from_yaml(yaml)
c

## ports

You can export the ports of any instance to the new `component_from_yaml` Component.

You will need to define a `ports` section in YAML

Lets expose all the ports from `mmi_long` into the new component.

Ports are exposed as `new_port_name: instance_name, port_name`

In [ ]:
import pp

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
placements:        
    mmi_long:
        port: W0
        x: 20
        y: 10
        mirror: True
        
ports:
    W0: mmi_long,E0
    W1: mmi_long,E1
    E0: mmi_long,W0
"""

c = pp.component_from_yaml(yaml)
c.show()
c.plot()

You can also define a mirror placement using a port

In [ ]:
import pp

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
placements:        
    mmi_long:
        x: 0
        y: 0
        mirror: W0
"""

c = pp.component_from_yaml(yaml)
c

In [ ]:
import pp

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
placements:        
    mmi_long:
        x: 0
        y: 0
"""

c = pp.component_from_yaml(yaml)
c.show()
c.plot()

In [ ]:
import pp

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
placements:        
    mmi_long:
        x: 0
        y: 0
        mirror: 25
ports:
    W0: mmi_long,E0
    W1: mmi_long,E1
    E0: mmi_long,W0
"""

c = pp.component_from_yaml(yaml)
c.show()
c.plot()

In [ ]:
import pp

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
placements:        
    mmi_long:
        port: W0
        x: 10
        y: 20
        rotation: 90
"""

c = pp.component_from_yaml(yaml)
c.show()
c.plot()

In [ ]:
import pp

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
placements:        
    mmi_long:
        port: W0
        x: 10
        y: 20
        rotation: 90
ports:
    N0: mmi_long,E0
    N1: mmi_long,E1
    S0: mmi_long,W0
"""

c = pp.component_from_yaml(yaml)
c.show()
c.plot()

In [ ]:
c.size_info.north

In [ ]:
c.size_info.east

## connections

You can connect any two instances by defining a `connections` section in the YAML file.

it follows the syntax.

`instance_source,port : instance_destination,port`

In [ ]:
import pp

yaml = """
instances:
    b:
      component: bend_circular
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 10
    mmi_short:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
placements:
    mmi_short:
        port: W0
        x: 10
        y: 20
connections:
    b,W0 : mmi_short,E1
    mmi_long,W0: b, N0

ports:
    W0: mmi_short,W0
"""


c = pp.component_from_yaml(yaml)
c.show()
c.plot()

In [ ]:
from omegaconf import OmegaConf
import io

In [ ]:
d = OmegaConf.load(io.StringIO(
"""
x: mmi_short,E0 10
dx: 10

"""
))
d

In [ ]:
d.keys()

**Relative port placing**

You can also place a component with respect to another instance port

You can also define an x and y offset with `dx` and `dy`

In [ ]:
import pp

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 10
    mmi_short:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5

placements:
    mmi_short:
        port: W0
        x: 0
        y: 0
    mmi_long:
        port: W0
        x: mmi_short,E1
        y: mmi_short,E1
        dx : 10
        dy: -10
"""


c = pp.component_from_yaml(yaml)
c.show()
c.plot()

**Cicular reference warning**

You have to be careful not doing a circuilar reference

In [ ]:
import pp

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 10
    mmi_short:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5

placements:
    mmi_short:
        port: W0
        x: mmi_long,E1
        y: mmi_long,E1
    mmi_long:
        port: W0
        x: mmi_short,E1
        y: mmi_short,E1
        dx : 10
        dy: 20
"""

pp.component_from_yaml(yaml) 

## routes

You can define routes between two instanes by defining a `routes` section in YAML

it follows the syntax

```YAML

routes:
    route_name:
        links:
            instance_source,port: instance_destination,port
        settings:  # for the route (optional)
            waveguide: strip
            width: 1.2

```

In [ ]:
import pp

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 10
    mmi_short:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
placements:
    mmi_long:
        x: 100
        y: 100
routes:
    optical:
        links:
            mmi_short,E1: mmi_long,W0
        settings:
            waveguide: strip_heater
"""


c = pp.component_from_yaml(yaml)
c.show()
c.plot()

You can **rotate** and instance specifying the angle in degrees

You can also define ports for the component

In [ ]:
import pp

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 10
    mmi_short:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
        
placements:
    mmi_long:
        rotation: 180
        x: 100
        y: 100
        
routes:
    optical:
        links:
            mmi_short,E1: mmi_long,E0
    
ports:
    E0: mmi_short,W0
    W0: mmi_long,W0
    
"""

c = pp.component_from_yaml(yaml)
c.show()
c.plot()

You can also access the routes in the newly created component

In [ ]:
r = c.routes['mmi_short,E1:mmi_long,E0']
r

As well as the instances

In [ ]:
c.instances

## instances, placements, connections, ports, routes

Lets combine all you learned so far.

You can define the netlist connections of a component by a netlist in YAML format

Note that you define the connections as `instance_source.port ->
instance_destination.port` so the order is important and therefore you can only
change the position of the `instance_destination`

For example, this coupler has the center coupling region at (100, 0)

In [ ]:
pp.components.coupler_symmetric()

In [ ]:
import pp

gap = 0.2
wg_width = 0.5
length = 10

yaml = f"""
instances:
    left:
      component: coupler_symmetric
      settings:
        gap: {gap}
        width: {wg_width}
    right:
      component: coupler_symmetric
      settings:
        gap: {gap}
        width: {wg_width}
    center:
      component: coupler_straight
      settings:
        gap: {gap}
        width: {wg_width}
        length: {length}

placements:
    center:
        x: 100
        y: 0
    left:
        mirror: True

connections:
    center,W0: left,W0
    right,W0: center,E0

ports:
    w0: left,E0
    w1: left,E1
    e0: right,E0
    e1: right,E1

"""

c = pp.component_from_yaml(yaml)
c

While this one has the sbend_left_coupler `sl` centered at (100, 0)

In [ ]:
import pp

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 10
    mmi_short:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
placements:
    mmi_long:
        x: 100
        y: 100
routes:
    route1:
        links:
            mmi_short,E1: mmi_long,W0
"""


c = pp.component_from_yaml(yaml)
c

You can **rotate** and instance specifying the angle in degrees

In [ ]:
import pp

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 10
    mmi_short:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
        
placements:
    mmi_long:
        rotation: 180
        x: 100
        y: 100
routes:
    mmi_connect:
        links:
            mmi_short,E1: mmi_long,E0
"""


c = pp.component_from_yaml(yaml)
c

You can also define ports for the component

In [ ]:
import pp

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 10
    mmi_short:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
        
placements:
    mmi_long:
        rotation: 180
        x: 100
        y: 100
        
routes:
    optical:
        links:
            mmi_short,E1: mmi_long,E0
    
ports:
    E0: mmi_short,W0
    W0: mmi_long,W0
"""

c = pp.component_from_yaml(yaml)
c

In [ ]:
c.routes

In [ ]:
r = c.routes['mmi_short,E1:mmi_long,E0']
r

In [ ]:
c.instances

In [ ]:
c.routes

You can define several routes that will be connected using `pp.routing.get_bundle` 

In [ ]:
import pp

sample_2x2_connections_solution = """
name:
    connections_2x2_problem

instances:
    mmi_bottom:
      component: mmi2x2
    mmi_top:
      component: mmi2x2

placements:
    mmi_top:
        x: 100
        y: 100

routes:
    optical:
        links:
            mmi_bottom,E0: mmi_top,W0
            mmi_bottom,E1: mmi_top,W1

"""

c = pp.component_from_yaml(sample_2x2_connections_solution)
c

You can also add custom component_factories to `pp.component_from_yaml`

In [ ]:
@pp.cell
def pad_new(size=(100, 100), layer=pp.LAYER.WG):
    c = pp.Component()
    compass = c << pp.components.compass(size=size, layer=layer)
    c.ports = compass.ports
    return c

c = pad_new(cache=False)
c

In [ ]:
import pp
from pp.components import LIBRARY

LIBRARY.register(pad_new)

sample_custom_component = """
name:
    connections_2x2_problem

instances:
    bot:
      component: pad_new
    top:
      component: pad_new

placements:
    top:
        x: 0
        y: 200
"""

c = pp.component_from_yaml(sample_custom_component, library=LIBRARY)
c

In [ ]:
sample_custom_component = """
name:
    custom_routes

instances:
    t:
      component: pad_array
      settings:
          port_list: ['S']
    b:
      component: pad_array

placements:
    t:
        x: 200
        y: 400
routes:
    electrical:
        settings:
            radius: 5
            waveguide: metal_routing
        links:
            t,S0: b,N0
            t,S5: b,N5
"""


c = pp.component_from_yaml(sample_custom_component)
c


Also, you can define route aliases, that have different settings and specify the route `factory` as a parameter as well as the `settings` for that particular route alias.

In [ ]:
import pp

sample_settings = """
name: sample_settings

instances:
    bl:
      component: pad
    tl:
      component: pad
    br:
      component: pad
    tr:
      component: pad

placements:
    tl:
        x: 0
        y: 200

    br:
        x: 400
        y: 400

    tr:
        x: 400
        y: 600

routes:
    optical_r100:
        settings:
            radius: 100
            waveguide: metal_routing
        links:
            tl,N: tr,N
    optical_r200:
        settings:
            radius: 200
            waveguide: metal_routing
        links:
            bl,S: br,E
"""
pp.component_from_yaml(sample_settings)

In [ ]:
sample_custom_component = """

instances:
    t:
      component: pad_array
      settings:
          port_list: ['S']
    b:
      component: pad_array

placements:
    t:
        x: 200
        y: 400
routes:
    optical:
        settings: 
            radius: 50
            waveguide: metal_routing
        links:
            t,S0: b,N0
            t,S5: b,N5
"""

pp.component_from_yaml(sample_custom_component)

In [ ]:
import pp

sample = """

instances:
    t:
      component: pad_array
      settings:
          port_list: ['S']
    b:
      component: pad_array

placements:
    t:
        x: 100
        y: 1000
routes:
    route1:
        routing_strategy: get_bundle_path_length_match
        settings: 
            extra_length: 500
            waveguide: metal_routing
        links:
            t,S0: b,N0
            t,S5: b,N4
"""

c = pp.component_from_yaml(sample)
print(c.routes['t,S0:b,N0'])
c

In [ ]:
import pp

sample = """
instances:
    t:
      component: pad_array
      settings:
          port_list: ['S']
    b:
      component: pad_array
      settings:
          port_list: ['N']

placements:
    t:
        x: -250
        y: 1000
routes:
    route1:
        routing_strategy: get_bundle_from_waypoints
        settings:
            waypoints:
                - [0, 300]
                - [400, 300]
                - [400, 400]
                - [-250, 400]
            auto_widen: False
        links:
            b,N0: t,S0
            b,N1: t,S1

"""

c = pp.component_from_yaml(sample)
c

In [ ]:
from omegaconf import OmegaConf
import io

d = OmegaConf.load(io.StringIO("""
way_points: 
    - [0,0]
    - [0, 600]
    - [-250, 600]
    - [-250, 1000]

demo: a
"""))

In [ ]:
d = OmegaConf.load(io.StringIO("way_points:  [[0,0], [0, 600], [-250, 600], [-250, 1000]]"))

In [ ]:
d

In [ ]:
import numpy as np

c = pp.Component('waypoints_sample')
route = pp.routing.get_route_from_waypoints(
    waypoints=np.array(d['way_points'])
)

c.add(route.references)
c

Note that you define the connections as `instance_source.port -> instance_destination.port` so the order is important and therefore you can only change the position of the `instance_destination`

**Custom factories**

You can leverage netlist defined components to define more complex circuits

In [ ]:
import pp

@pp.cell
def mzi_custom(length_x=0):
    netlist = f"""
instances:
    mzi:
        component: mzi_phase_shifter
        
    pads:
        component: pad_array
        settings:
            n: 2
            port_list: 
                - S

placements:
    mzi:
        x: 0
    pads:
        y: 200
        x:  50
ports:
    W0: mzi,W0
    E0: mzi,E0
    
    
routes:
    electrical:
        links:
            mzi,N0: pads,S0
            mzi,N1: pads,S1
            
        settings:
            waveguide: metal_routing

"""
    return pp.component_from_yaml(netlist)


c = mzi_custom(length_x=10, cache=False)
c.show()
c.plot()

In [ ]:
c = pp.components.mzi()

In [ ]:
c.plot_netlist()

In [ ]:
n = c.get_netlist()

In [ ]:
print(c.get_netlist_yaml())

## get_netlist (Component -> YAML)

Any component exports its netlist `get_netlist` and returns an `OmegaConf` dict that can be easily converted into JSON and YAML.

While `component_from_yaml` converts YAML -> Component

`get_netlist` converts Component -> YAML

In [ ]:
import io
from omegaconf import OmegaConf
import pp

In [ ]:
c = pp.components.mzi()
c

In [ ]:
c.plot_netlist()

In [ ]:
netlist = c.get_netlist()

In [ ]:
c.write_netlist('mzi.yml', full_settings=True)

In [ ]:
n = OmegaConf.load('mzi.yml')

In [ ]:
i = list(n['instances'].keys())
i

In [ ]:
n['instances'][i[0]]['settings']

In [ ]:
import pp
from omegaconf import OmegaConf
import pathlib

c1 = pp.component_from_yaml('mzi.yml')
c1

In [ ]:
n = c1.get_netlist(full_settings=True)
connections = n['connections']
len(connections)

## Plot netlist

You can plot the netlist of components.

Every gdsfactory component can either be defined by its netlist or using layout friendly functions such as component sequence to define it and then `get_netlist()` method.

Connections are determined by extracting all the ports of a component, and asuming that ports with the same (x, y) are connected.

 When you do `get_netlist()` for a component it will only show connections for the instances that belong to that component (it trims the netlist). So despite having  a lot of connections, it will show only the meaningful connections for that component. For example, a ring has a ring_coupler. but if you want to digg deeper, the connections that made that ring coupler are still available.

In [ ]:
import pp

In [ ]:
c = pp.components.mzi()
c

In [ ]:
c = pp.components.mzi()
n = c.get_netlist()
print(c.get_netlist_yaml())

In [ ]:
c.plot_netlist()

In [ ]:
n.keys()

In [ ]:
import pp

yaml =  """
instances:

    mmi1x2_12_0:
        component: mmi1x2

    bend_circular_R10p00_32_4:
      component: bend_circular
    
    straight_L1p00_35_11:
        component: straight
        settings:
            length: 10
            layer: [2, 0]

connections:
    bend_circular_R10p00_32_4,W0: mmi1x2_12_0,E1
    straight_L1p00_35_11,W0: bend_circular_R10p00_32_4,N0
"""

c = pp.component_from_yaml(yaml)
c.show()
c.plot()
c.name = "mmi_with_bend_circular"
print(c.name)

In [ ]:
n = c.get_netlist()

In [ ]:
print(c.get_netlist_yaml())

In [ ]:
n['connections']

In [ ]:
c.plot_netlist()

In [ ]:
c = pp.components.mzi()
c.plot()
c.plot_netlist()

In [ ]:
c = pp.components.ring_single()
c

In [ ]:
c.plot_netlist()

In [ ]:
c = pp.components.ring_double()
c

In [ ]:
c.plot_netlist()

In [ ]:
import pp

c = pp.components.ring_single()
c

In [ ]:
c.plot_netlist()

In [ ]:
c = pp.components.ring_double()
c

In [ ]:
c.plot_netlist()

In [ ]:
print(c.get_netlist_yaml())

In [ ]:
c = pp.components.mzi()
c

In [ ]:
c.plot_netlist()

In [ ]:
c = pp.components.mzit()
c

In [ ]:
c.plot_netlist()

In [ ]:
c = pp.components.mzi_lattice()
c

In [ ]:
import pp
coupler_lengths = [10, 20, 30]
coupler_gaps = [0.1, 0.2, 0.3]
delta_lengths = [10, 100]

c = pp.components.mzi_lattice(coupler_lengths=coupler_lengths, coupler_gaps=coupler_gaps, delta_lengths=delta_lengths)
c

In [ ]:
print(c.get_netlist_yaml())

In [ ]:
c.plot_netlist()

In [ ]:
coupler_lengths = [10, 20, 30, 40]
coupler_gaps = [0.1, 0.2, 0.4, 0.5]
delta_lengths = [10, 100, 200]

c = pp.components.mzi_lattice(coupler_lengths=coupler_lengths, coupler_gaps=coupler_gaps, delta_lengths=delta_lengths)
c

In [ ]:
n = c.get_netlist()

In [ ]:
c.plot_netlist()